<a href="https://colab.research.google.com/github/huzaifa003/Assignment09/blob/main/SwimTransformerSparsaComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/KrisnaPinasthika/SparseSwin

Cloning into 'SparseSwin'...
remote: Enumerating objects: 657, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 657 (delta 14), reused 54 (delta 11), pack-reused 596 (from 1)
Receiving objects: 100% (657/657), 1.11 GiB | 16.66 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Updating files: 100% (548/548), done.


In [ ]:
%cd SparseSwin

In [ ]:
!pip install timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
%cd SparseSwin

/content/SparseSwin


In [ ]:
!python train_cifar.py --swin_type tiny \
                      --img_res 224 \





[TRAIN] Total : 782 | type : tiny | Regularization : None with lamda : 0.0
Epoch 1/100
  [200/782] Loss: 2.0311 Acc : 0.2398
  [400/782] Loss: 1.9791 Acc : 0.2607
  [600/782] Loss: 1.9187 Acc : 0.2865
  [782/782] Loss: 1.8747 Acc : 0.3052
Loss: 1.8747 Acc : 0.3052
[TEST] Total : 157 | type : tiny
[Model : tiny] Loss: 1.6510 Acc : 0.3834

Epoch 2/100
  [200/782] Loss: 1.6563 Acc : 0.3931
  [400/782] Loss: 1.6244 Acc : 0.4041
  [600/782] Loss: 1.5938 Acc : 0.4189
  [782/782] Loss: 1.5710 Acc : 0.4290
Loss: 1.5710 Acc : 0.4290
[TEST] Total : 157 | type : tiny
[Model : tiny] Loss: 1.3973 Acc : 0.4997

Epoch 3/100
  [200/782] Loss: 1.4571 Acc : 0.4749
  [400/782] Loss: 1.4375 Acc : 0.4844
  [600/782] Loss: 1.4150 Acc : 0.4909
  [782/782] Loss: 1.4105 Acc : 0.4919
Loss: 1.4105 Acc : 0.4919
[TEST] Total : 157 | type : tiny
[Model : tiny] Loss: 1.2342 Acc : 0.5583

Epoch 4/100
  [200/782] Loss: 1.3424 Acc : 0.5170
  [400/782] Loss: 1.3315 Acc : 0.5182
  [600/782] Loss: 1.3259 Acc : 0.5205
  [7